In [118]:
#Imports
import matplotlib.pyplot as plt
import numpy as np

from utils import read_input_file
from sample_gen import sample_prior,descritize_space
from data_gen import generate_data
from like_models import compute_loglikes

In [119]:
#Load Configuration
nlpts, ndata, lat_range, long_range, depth_range, sensors = read_input_file('inputs.dat')

In [120]:
#Sample prior some how to generate events that we will use to generate data
theta_data = sample_prior(lat_range,long_range, depth_range, nlpts)

In [121]:
#Generate Hypothetical Datasets
dataz = np.zeros([nlpts*ndata,np.int(sensors.shape[0]*sensors[0,3])])
for ievent in range(0,nlpts):
    theta = theta_data[ievent,:]
    dataz[(ievent*ndata):((ievent+1)*ndata),:] = generate_data(theta,sensors,ndata)

In [122]:
#Define the event space descritization
theta_space = descritize_space(lat_range,long_range, depth_range, nlpts)

In [123]:
#Distribute data and event space to processes
#Compute likelhood of each event given dataset
loglikes = np.zeros([nlpts*ndata,nlpts])
for ievent in range(0,nlpts):
    theta = theta_space[ievent,:]
    
    #compute likelihoods
    loglikes[:,ievent] = compute_loglikes(theta,sensors,dataz)
    
#Collect data


In [124]:
#Combine data to compute posterior and KL divergence
ig = np.zeros(nlpts*ndata)
for idata in range(0,nlpts*ndata):
    loglike = loglikes[idata,:];
    probs = np.exp(loglike - np.max(loglike))/np.sum(np.exp(loglike - np.max(loglike)))
    logprobs = (loglike - np.max(loglike)) - np.log(np.sum(np.exp(loglike - np.max(loglike))))
    ig[idata] = np.sum(probs*(logprobs+np.log(probs.size)))

In [125]:
#Average
np.mean(ig)

0.79982953277006597